# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
# Import seaborn, numpy, pandas and stat
import numpy as np
import pandas as pd

from scipy import stats
import seaborn as sns

In [4]:
df = pd.read_csv("homepage_actions.csv")
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [92]:
df["id"].value_counts()

750053    2
389447    2
283040    2
918794    2
208714    2
         ..
467818    1
629924    1
820336    1
424916    1
461199    1
Name: id, Length: 6328, dtype: int64

In [15]:
df.groupby("id")["action"].value_counts()

id      action
182988  view      1
182994  click     1
        view      1
183089  view      1
183136  view      1
                 ..
937108  view      1
937139  click     1
        view      1
937217  click     1
        view      1
Name: action, Length: 8188, dtype: int64

In [30]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [80]:
#Assumption, the data collection is correct...looking for anomlies in data
#Only looking at data that has an id

#Numbers of viewers
    #The length of the ids who viewed (only one time)
viewID = df[df["action"] == "view"]["id"].unique()
NumV =len(viewID)
#Numbers of clickers
    #The length of the ids who clicked (only one time)
clickID = df[df["action"] == "click"]["id"].unique()
NumC =len(clickID)
print ("viewID: ",NumV, "clickID: ",NumC)

viewID:  6328 clickID:  1860


In [79]:
#Number of views who didn't click
    #The difference between the length views and clicks
print ("The number of views who didnt click:", NumV-NumC)
#Numbers of clickers who didn't view
    #The difference between non views and clicks
NonV = 8188 - NumV
print ("The number of views who didnt click:", NumC-NonV)

The number of views who didnt click: 4468
The number of views who didnt click: 0


In [89]:
#Looking to see how some of the ids are allocated...some ids are in the control or exp group more than once
df.groupby("id")["group"].value_counts()

id      group     
182988  experiment    1
182994  control       2
183089  control       1
183136  experiment    1
183141  experiment    2
                     ..
937003  control       1
937073  control       1
937108  control       1
937139  experiment    2
937217  control       2
Name: group, Length: 6328, dtype: int64

In [134]:
#Is there any overlap between the control and experiment groups? * 
#If so, how do you plan to account for this in your experimental design?
#We do not want someone with the same Id to be in both the control and exprimental group
#Check the solution to do it using sets

len(df[(df["group"] == "experiment")& (df["group"] == "control")]["id"])
#There is no overlap between experiments and control group


0

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [ ]:
#The number of clicks would determine if the exp was more effective than the control group
#We are comparing category of exp and control and the category of clicks and views

#Get a count of (non views, views and clicks) by experiment and control group and put in table the number

In [ ]:
#Hypothesis 

#Null Hypothesis: The exp homepage was no more effective than that of the control group
#Alt Hypothesis: The exp was more effective than that of the control group

In [ ]:
#We want to take understand the amount of clicks for each group
#We want to take statistical measures on on the data
    # Sample Size - Control, Experiment; Clicks - Control, Experiment; mean- c,e and std, e,v
    #We want to  get the p value

    

In [128]:
#Experiment groups with one experiment trial and no control
expnum = len(df[(df["group"] == "experiment") & (df["group"] != "control")]["id"].unique())
expnum

2996

In [129]:
#Control groups with one control trial and no experiment
connum = len(df[(df["group"] != "experiment") & (df["group"] == "control")]["id"].unique())
connum

3332

In [153]:
#Get the total amount of clicks from the exp group and the experiment array
expclick_len = len(df[(df["group"] == "experiment") & (df["group"] != "control") & (df["action"] == "click") ]["id"].unique())
exp_click = df[(df["group"] == "experiment") & (df["group"] != "control") & (df["action"] == "click") ]["id"].unique()

In [159]:
#Get the total amount of clicks from the control group and the control array
conclick_len = len(df[(df["group"] != "experiment") & (df["group"] == "control") & (df["action"] == "click") ]["id"].unique())
con_click = df[(df["group"] != "experiment") & (df["group"] == "control") & (df["action"] == "click") ]["id"].unique()

In [173]:
print("Sample sizes--> Control Group: {} Experiment Group: {}".format(connum, expnum))
print("Clicks--> Control Group: {} Experiment Group: {}".format(conclick_len, expclick_len))
print("Mean Click Rate--> Control Group: {} Experiment Group: {}".format(conclick_len/connum, expclick_len/expnum))

Sample sizes--> Control Group: 3332 Experiment Group: 2996
Clicks--> Control Group: 932 Experiment Group: 928
Mean Click Rate--> Control Group: 0.2797118847539016 Experiment Group: 0.3097463284379172


In [215]:
#Need a Control and Experiment dataframe to get p-value...need a dataframe with more than IDs like you had previously

df['count'] = 1

control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)


In [257]:
#import flatiron_stats as fs - doesn't load

t_stat, p_value = stats.ttest_ind(control.click, experiment.click,equal_var=False, nan_policy='omit') 
one_side_p = p_value/2
one_side_p

0.004466402814337101

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [258]:
con_rate = control.click.mean()
con_rate

0.2797118847539016

In [246]:
#1 clicks = control rate * num of experiments
exp_rate = experiment.click.mean()
con_rate = control.click.mean()
elen = len(experiment)
expected_e_click = con_rate * elen
expected_e_click



838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [244]:
#Variance = n * p(1-p) # std sqr(var)
#Use Zscore (hint below)

n = elen
p = con_rate

var = n*p*(1-p)
std = np.sqrt(var)
std


24.568547907005815

In [245]:
experiment.click.sum()

928.0

In [253]:
#Calculate Z score - Std known, and greater than 30 sample size
# z = (observed value(928) - mean of sample (calc val 838) )/std of sample (24.5)

zsc = (experiment.click.sum() - expected_e_click)/std
print ("z score:",zsc)

z score: 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [256]:
#Pvalue using normal distribution

p_value = stats.norm.sf(zsc)
print("pvalue:", p_value)

pvalue: 0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Yes, my intial test had the p value less than 0.05 which supports the experiment group was more effective than the control group rejecting the null hypothesis.  This validation's test of the p value was a lot less than the initial p value but still supports the same claim.  The effect size would help us understand by how much.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.